In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_epochs = 10
batch_size = 64
learning_rate = 0.001

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
model = LeNet().to(device)
model.apply(weights_init)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(model, criterion, optimizer, train_loader, num_epochs):
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = correct / total * 100
        return accuracy

In [ ]:
train_model(model, criterion, optimizer, train_loader, num_epochs)

In [ ]:
baseline_accuracy = test_model(model, test_loader)
print(f"Baseline Accuracy: {baseline_accuracy:.2f}%")

In [ ]:
# Pruning rates for evaluation
pruning_rates = [0.2, 0.5, 0.8]

for prune_rate in pruning_rates:
        # Prune nodes using PCA
    def prune_nodes_PCA(model, prune_ratio):
        # Extracting weights
        weights = torch.cat([param.view(-1) for param in model.parameters()]).cpu().detach().numpy()

        # Print weights for debugging
        print("Weights before PCA pruning:", weights)

        # Applying PCA
        pca = PCA(n_components=1)
        pca.fit(weights.reshape(-1, 1))
        explained_variance_ratio = np.sum(pca.explained_variance_ratio_)

        # Print explained variance for debugging
        print("Explained Variance Ratio (PCA):", explained_variance_ratio)

        node_threshold = np.percentile(np.abs(weights), prune_ratio * 100)

        # Print node threshold for debugging
        print("Node Threshold (PCA):", node_threshold)

        # Prune nodes
        for param in model.parameters():
            param.data[torch.abs(param.data) < node_threshold] = 0

        return explained_variance_ratio

    explained_variance_ratio_nodes_pca = prune_nodes_PCA(model, prune_rate)
    accuracy_nodes_pruned_pca = test_model(model, test_loader)
    print(f"PCA Pruning Rate: {prune_rate}, Accuracy after pruning nodes: {accuracy_nodes_pruned_pca:.2f}%")
    print(f"Explained Variance Ratio (Nodes - PCA): {explained_variance_ratio_nodes_pca:.2f}")

    # Prune connections using PCA
    def prune_connections_PCA(model, prune_ratio):
        # Extracting weights
        weights = torch.cat([param.view(-1) for param in model.parameters()]).cpu().detach().numpy()

        # Print weights for debugging
        print("Weights before PCA pruning:", weights)

        # Applying PCA
        pca = PCA(n_components=1)
        pca.fit(weights.reshape(-1, 1))
        explained_variance_ratio = np.sum(pca.explained_variance_ratio_)

        # Print explained variance for debugging
        print("Explained Variance Ratio (PCA):", explained_variance_ratio)

        weight_threshold = np.percentile(np.abs(weights), prune_ratio * 100)

        # Print node threshold for debugging
        print("Weights Threshold (PCA):", weight_threshold)

        # Prune connections
        for param in model.parameters():
            param.data[torch.abs(param.data) < weight_threshold] = 0

        return explained_variance_ratio

    explained_variance_ratio_connections_pca = prune_connections_PCA(model, prune_rate)
    accuracy_connections_pruned_pca = test_model(model, test_loader)
    print(f"PCA Pruning Rate: {prune_rate}, Accuracy after pruning connections: {accuracy_connections_pruned_pca:.2f}%")
    print(f"Explained Variance Ratio (Connections - PCA): {explained_variance_ratio_connections_pca:.2f}")